<a href="https://colab.research.google.com/github/adalbertii/Modele-regresyjne/blob/main/pipeline-boston-house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Model regresyjny**

Oparty na sieciach neuronowych

1.   Ocena modeli sieci neuronowych z punktu widzenia ich wewnętrznej topologii
2.   Badany jest równiez wpływ warstwy standaryzującej
3.   Prezentowanana jest koncepcja budowy PIPELINE

In [ ]:
! pip install Keras

In [ ]:
! pip install scikeras

In [2]:

from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [6]:
#url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
#datframe = read_csv(url)
dataframe = read_csv('boston.csv')

In [7]:
datframe.head()


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [8]:
datframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [9]:
# konwersja dataframe do typu tablicowego

dataset = datframe.values

In [10]:
dataset.shape

(506, 14)

In [12]:
# podział zbioru danych na zbirr ze zmiennymi objaśniajacymi [X] i zmienną objasnianą [Y]
X = dataset[:,:-1]
Y = dataset[:,-1]

In [13]:
Y.shape

(506,)

In [15]:
X.shape

(506, 13)

In [17]:
# definicja procedury przeznaczonej do budowy i kompilacji modelu sieci neuronowej
def baseline_model():

 model = Sequential()
 # parametr input_sape jest taki sam jak kształt tablicy X
 model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))

 # kompilacja modelu
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


**Ewaluacja modelu**
oparatA na walidacji krzyżowej , z podziałem zbioru na 10 podzbiorów

In [18]:

estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=1)
kfold = KFold(n_splits=10)


In [ ]:
results = cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error')


In [20]:
print("Model podstawowy: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -34.88 (26.39) MSE


**Budowa modelu z warstwą standaryzującą dane wejściowe do modelu **

In [21]:
# DEFINICJA PROCEDURY BUDOWY MODELU
def baseline_model():

 model = Sequential()
 model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))
 # kompilacja  modelu
 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [24]:
# budowa pipeline ze standaryzacją danych wejściowych i budową, uczeniem i ewaluacją modelu

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=baseline_model, epochs=50, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)


In [ ]:
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Model podstawowy z warstwą standaryzującą: %.2f (%.2f) MSE" % (results.mean(), results.std()))

**Dostrajanie topologii sieci neuronowej**

Model sieci neuronowej można zoptymalizować pod wieloma względami.

Być może punktem największego wpływu jest struktura samej sieci, w tym liczba warstw i liczba neuronów w każdej warstwie.

W tej sekcji zostaną ocenione dwie dodatkowe topologie sieci w celu dalszej poprawy wydajności modelu.

Przyjrzymy się zarówno głębszej, jak i szerszej topologii sieci.



**Ocena głębszej topologii sieci**

Jednym ze sposobów poprawy wydajności sieci neuronowej jest dodanie większej liczby warstw. Może to pozwolić modelowi na wyodrębnienie i ponowne połączenie cech wyższego rzędu osadzonych w danych.

W tej sekcji oceniony zostanie efekt dodania do modelu jeszcze jednej warstwy ukrytej. Jest to tak proste, jak zdefiniowanie nowej funkcji do tworzenia tego głębszego modelu, skopiowanego z powyższego modelu bazowego. Następnie można wstawić nową linię po pierwszej warstwie ukrytej - w tym przypadku z około połową liczby neuronów

In [26]:
# procedura budowy głębszej sieci neuronowej
def larger_model():


 model = Sequential()
 model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(6, kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))

 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


In [27]:
# evaluacja  głębszego modelu z warstwą standaryzującą
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)


In [28]:
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Głębszy model z warstwą standaryzującą : %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -23.80 (25.13) MSE


**Ocena szerszej topologii sieci**

Innym podejściem do zwiększenia możliwości reprezentacyjnych modelu jest stworzenie szerszej sieci.

W tej sekcji ocenimy efekt zachowania płytkiej architektury sieci i niemal podwojenia liczby neuronów w jednej warstwie ukrytej.

 W tym przypadku liczba neuronów w warstwie ukrytej zostanie zwiększona z 13 do 20 w porównaniu do modelu bazowego.

In [30]:

def wider_model():

 model = Sequential()
 model.add(Dense(20, input_shape=(13,), kernel_initializer='normal', activation='relu'))
 model.add(Dense(1, kernel_initializer='normal'))

 model.compile(loss='mean_squared_error', optimizer='adam')
 return model


In [31]:
# evaluacja szerszego modelu
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)


In [32]:
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Szerszy model z warstwą standaryzującą: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -21.54 (23.94) MSE
